In [ ]:
from IPython.display import display, Markdown, clear_output
import utils
import ipywidgets as ipw
import widgets
import json

# Sample preparation

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

In [ ]:
class ActionsWidgets(ipw.VBox):
    def __init__(self, history = True, is_protocol = False):
        super().__init__()
        self.accordion = ipw.Accordion()
        if history:
            self.children = [self.accordion]
        else:
            self.accordion_items = {}
            if is_protocol:
                self.children = [self.accordion]
            else:
                self.add_button = utils.Button(
                    description = 'Add action', disabled = False, button_style = 'success',
                    tooltip = 'Add action', layout = ipw.Layout(width = '150px', height = '25px')
                )
                self.add_button.on_click(self.add_action)
                self.children = [self.accordion, self.add_button]
    
    def add_action(self, b):
        # Component
        component_selector = widgets.ObjectSelectionWidget("Component", disabled = False)
        component_selector.load_dropdown_box()
        
        # Process Step properties
        properties_widgets = widgets.ObjectPropertiesWidgets("Action")
        properties_widgets.get_properties_widgets()
        
        remove_button = ipw.Button(description = "Remove action", button_style = 'danger')
        action_widgets = ipw.VBox([component_selector, properties_widgets, remove_button])
        
        def update_item_title(change):
            item_idx, _ = self.accordion_items[action_widgets]
            new_title = properties_widgets.properties_widgets_detailed_dict["$name"].value
            self.accordion.set_title(item_idx, new_title)
            self.accordion_items[action_widgets] = [item_idx, new_title]
        
        properties_widgets.properties_widgets_detailed_dict["$name"].observe(update_item_title, names = 'value')
        
        def remove_action(b):
            self.accordion_items.pop(action_widgets)
            i = 0
            for item, item_info in self.accordion_items.items():
                self.accordion_items[item] = [i, item_info[1]]
                self.accordion.set_title(i, item_info[1])
                i += 1
            self.accordion.set_title(i, "")
            self.accordion.children = list(self.accordion_items.keys())
        
        remove_button.on_click(remove_action)
        self.accordion_items[action_widgets] = [len(self.accordion_items), ""]
        self.accordion.children = list(self.accordion_items.keys())
        
    def add_action_template(self, template):
        for action_template in template:
            # Component
            component_selector = widgets.ObjectSelectionWidget("Component", disabled = False)
            component_selector.load_dropdown_box()
        
            # Process Step properties
            properties_widgets = widgets.ObjectPropertiesWidgets("Action")
            properties_widgets.get_properties_widgets()
            properties_widgets.properties_widgets_detailed_dict["$name"].value = action_template["$name"]
            properties_widgets.properties_widgets_detailed_dict["action_type"].value = action_template["action_type"]
            
            action_widgets = ipw.VBox([component_selector, properties_widgets])
            
            def update_item_title(change):
                item_idx, _ = self.accordion_items[action_widgets]
                new_title = properties_widgets.properties_widgets_detailed_dict["$name"].value
                self.accordion.set_title(item_idx, new_title)
                self.accordion_items[action_widgets] = [item_idx, new_title]
            
            properties_widgets.properties_widgets_detailed_dict["$name"].observe(update_item_title, names = 'value')
            self.accordion.set_title(len(self.accordion_items), action_template["$name"])
            self.accordion_items[action_widgets] = [len(self.accordion_items), ""]
            self.accordion.children = list(self.accordion_items.keys())
            
    def load_accordion(self, openbis_session, process_actions):
        accordion_widgets = []
        for process_action_identifier in process_actions:
            process_action_data = utils.get_openbis_object_data(openbis_session, process_action_identifier)
            process_action_title = process_action_data["props"]["$name"] + " (" + process_action_data["registration_date"] + ")"
            self.accordion.set_title(len(accordion_widgets), process_action_title)
            properties_widgets = widgets.ObjectPropertiesWidgets("Action", disabled = True)
            properties_widgets.get_properties_widgets()
            
            utils.load_widget_values(openbis_session, properties_widgets.properties_widgets_detailed_dict, process_action_data["props"])
            
            component_selector = widgets.ObjectSelectionWidget("Component", disabled = True)
            component_selector.load_dropdown_box()
            
            for parent_identifier in process_action_data["parents"]:
                parent_data = utils.get_openbis_object_data(openbis_session, parent_identifier)
                if parent_data["type"] == "COMPONENT":
                    component_selector.dropdown.value = parent_data["permId"]
            
            action_widgets = ipw.VBox([component_selector, properties_widgets])
            accordion_widgets.append(action_widgets)
        self.accordion.children = accordion_widgets

class ObservablesWidgets(ipw.VBox):
    def __init__(self, history = True, is_protocol = False):
        super().__init__()
        self.accordion = ipw.Accordion()
        if history:
            self.children = [self.accordion]
        else:
            self.accordion_items = {}
            if is_protocol:
                self.children = [self.accordion]
            else:
                self.add_button = utils.Button(
                    description = 'Add observable', disabled = False, button_style = 'success', 
                    tooltip = 'Add observable', layout = ipw.Layout(width = '150px', height = '25px')
                )
                self.add_button.on_click(self.add_observable)
                self.children = [self.accordion, self.add_button]
            
    def add_observable(self, b):
        # Component
        component_selector = widgets.ObjectSelectionWidget("Component", disabled = False)
        component_selector.load_dropdown_box()
        
        # Process Step properties
        properties_widgets = widgets.ObjectPropertiesWidgets("Observable")
        properties_widgets.get_properties_widgets()
        
        remove_button = ipw.Button(description = "Remove observable", button_style = 'danger')
        observable_widgets = ipw.VBox([component_selector, properties_widgets, remove_button])
        
        def update_item_title(change):
            item_idx, _ = self.accordion_items[observable_widgets]
            new_title = properties_widgets.properties_widgets_detailed_dict["$name"].value
            self.accordion.set_title(item_idx, new_title)
            self.accordion_items[observable_widgets] = [item_idx, new_title]
        
        properties_widgets.properties_widgets_detailed_dict["$name"].observe(update_item_title, names = 'value')
        
        def remove_observable(b):
            self.accordion_items.pop(observable_widgets)
            i = 0
            for item, item_info in self.accordion_items.items():
                self.accordion_items[item] = [i, item_info[1]]
                self.accordion.set_title(i, item_info[1])
                i += 1
            self.accordion.set_title(i, "")
            self.accordion.children = list(self.accordion_items.keys())
        
        remove_button.on_click(remove_observable)
        self.accordion_items[observable_widgets] = [len(self.accordion_items), ""]
        self.accordion.children = list(self.accordion_items.keys())

    def add_observable_template(self, template):
        for observable_template in template:
            # Process Step properties
            properties_widgets = widgets.ObjectPropertiesWidgets("Observable")
            properties_widgets.get_properties_widgets()
            properties_widgets.properties_widgets_detailed_dict["$name"].value = observable_template["$name"]
            
            # Component
            component_selector = widgets.ObjectSelectionWidget("Component", disabled = False)
            component_selector.load_dropdown_box()
            
            observable_widgets = ipw.VBox([component_selector, properties_widgets])
            
            def update_item_title(change):
                item_idx, _ = self.accordion_items[observable_widgets]
                new_title = properties_widgets.properties_widgets_detailed_dict["$name"].value
                self.accordion.set_title(item_idx, new_title)
                self.accordion_items[observable_widgets] = [item_idx, new_title]
            
            properties_widgets.properties_widgets_detailed_dict["$name"].observe(update_item_title, names = 'value')
            self.accordion.set_title(len(self.accordion_items), observable_template["$name"])
            self.accordion_items[observable_widgets] = [len(self.accordion_items), ""]
            self.accordion.children = list(self.accordion_items.keys())
            
    def load_accordion(self, openbis_session, process_observables):
        accordion_widgets = []
        
        for process_observable_identifier in process_observables:
            process_observable_data = utils.get_openbis_object_data(openbis_session, process_observable_identifier)
            process_observable_title = process_observable_data["props"]["$name"] + " (" + process_observable_data["registration_date"] + ")"
            self.accordion.set_title(len(accordion_widgets), process_observable_title)
            properties_widgets = widgets.ObjectPropertiesWidgets("Observable", disabled = True)
            properties_widgets.get_properties_widgets()
            
            utils.load_widget_values(openbis_session, properties_widgets.properties_widgets_detailed_dict, process_observable_data["props"])
            
            component_selector = widgets.ObjectSelectionWidget("Component", disabled = True)
            component_selector.load_dropdown_box()
            
            for parent_identifier in process_observable_data["parents"]:
                parent_data = utils.get_openbis_object_data(openbis_session, parent_identifier)
                if parent_data["type"] == "COMPONENT":
                    component_selector.dropdown.value = parent_data["permId"]
            
            observable_widgets = ipw.VBox([component_selector, properties_widgets])
            accordion_widgets.append(observable_widgets)
        self.accordion.children = accordion_widgets

class ProcessStepWidgets(ipw.VBox):
    def __init__(self):
        super().__init__()
        self.protocol_identifier = None
    
    def set_process_step(self, process_data, actions, observables):
        if process_data:
            self.process_data = process_data
            self.title = self.process_data["props"]["$name"] + " (" + self.process_data["registration_date"] + ")"
            
        self.actions = actions
        self.observables = observables
        self.actions_label = ipw.HTML("<b><span style='font-size:14px;'>Actions</span></b>")
        self.observables_label = ipw.HTML("<b><span style='font-size:14px;'>Observables</span></b>")
        self.children = [self.actions_label, self.actions, self.observables_label, self.observables]
    
    def find_protocol(self, openbis_session):
        for process_parent_identifier in self.process_data["parents"]:
            process_parent_data = utils.get_openbis_object_data(openbis_session, process_parent_identifier)
            if process_parent_data["type"] == "SAMPLE_PROCESS":
                self.protocol_identifier = process_parent_data["permId"]
                break

class ProtocolProcessWidgets(ipw.VBox):
    def __init__(self):
        super().__init__()
        self.accordion = ipw.Accordion()
        self.accordion_items = {}
        self.children = [self.accordion]
    
    def set_protocol_process_steps(self, template):
        for process_step_template in template:
            # Process Step properties
            properties_widgets = widgets.ObjectPropertiesWidgets("SampleProcess")
            properties_widgets.get_properties_widgets()
            properties_widgets.properties_widgets_detailed_dict["$name"].value = process_step_template["$name"]
            
            # Actions
            actions_widgets = ActionsWidgets(history = False, is_protocol = True)
            actions_widgets.add_action_template(process_step_template["actions"])
            
            # Observations
            observables_widgets = ObservablesWidgets(history = False, is_protocol = True)
            observables_widgets.add_observable_template(process_step_template["observables"])
            
            # Process Step widgets
            process_step_widgets = ProcessStepWidgets()
            process_step_widgets.set_process_step(None, actions_widgets, observables_widgets)
            
            process_step_widgets = ipw.VBox([properties_widgets, process_step_widgets])
            
            def update_item_title(change):
                item_idx, _ = self.accordion_items[process_step_widgets]
                new_title = properties_widgets.properties_widgets_detailed_dict["$name"].value
                self.accordion.set_title(item_idx, new_title)
                self.accordion_items[process_step_widgets] = [item_idx, new_title]
            
            properties_widgets.properties_widgets_detailed_dict["$name"].observe(update_item_title, names = 'value')
            self.accordion.set_title(len(self.accordion_items), process_step_template["$name"])
            self.accordion_items[process_step_widgets] = [len(self.accordion_items), process_step_template["$name"]]
            self.accordion.children = list(self.accordion_items.keys())

class SamplePreparationWidgets(ipw.VBox):
    def __init__(self, history = True):
        super().__init__()
        self.accordion = ipw.Accordion()
        
        if history:
            self.children = [self.accordion]
        else:
            self.add_process_button = utils.Button(
                description = 'Add protocol process', disabled = False, button_style = 'success', 
                tooltip = 'Add process with protocol', layout = ipw.Layout(width = '150px', height = '25px')
            )
            
            self.add_process_step_button = utils.Button(
                description = 'Add process step', disabled = False, button_style = 'success', 
                tooltip = 'Add process step', layout = ipw.Layout(width = '150px', height = '25px')
            )
            
            self.add_processes_buttons = ipw.HBox([self.add_process_button, self.add_process_step_button])
            
            self.add_process_button.on_click(self.add_protocol_process)
            self.add_process_step_button.on_click(self.add_process_step)
            self.accordion_items = {}
            self.children = [self.accordion, self.add_processes_buttons]

    def create_process_step_widgets(self, openbis_session, sample_processes):
        accordion_items = []
        protocol_process_items = {}
        for process_identifier, sample_process in sample_processes.items():
            process_data = utils.get_openbis_object_data(OPENBIS_SESSION, process_identifier)
            
            # Actions
            process_actions_accordion = ActionsWidgets()
            process_actions_accordion.load_accordion(OPENBIS_SESSION, sample_process["actions"])
            
            # Observables
            process_observables_accordion = ObservablesWidgets()
            process_observables_accordion.load_accordion(OPENBIS_SESSION, sample_process["observables"])
            
            # Process Step Widgets
            process_step_widgets = ProcessStepWidgets()
            process_step_widgets.set_process_step(process_data, process_actions_accordion, process_observables_accordion)
            process_step_widgets.find_protocol(OPENBIS_SESSION)
            
            if process_step_widgets.protocol_identifier:
                process_step_details = {"name": process_data["props"]["$name"], "registration_date": process_data["registration_date"], "items": process_step_widgets}
                protocol_process_items.setdefault(process_step_widgets.protocol_identifier, []).append(process_step_details)
            else:
                self.accordion.set_title(len(accordion_items), process_step_widgets.title)
                accordion_items.append(process_step_widgets)
        
        for protocol_process_identifier, protocol_process_steps_details in protocol_process_items.items():
            protocol_process_data = utils.get_openbis_object_data(OPENBIS_SESSION, protocol_process_identifier)
            protocol_process_accordion = ipw.Accordion()
            protocol_process_accordion_list = []
            for protocol_process_step_details in protocol_process_steps_details:
                protocol_process_title = protocol_process_step_details["name"] + " (" + protocol_process_step_details["registration_date"] + ")"
                protocol_process_accordion.set_title(len(protocol_process_accordion_list), protocol_process_title)
                protocol_process_accordion_list.append(protocol_process_step_details["items"])
            protocol_process_accordion.children = protocol_process_accordion_list
            
            process_step_title = protocol_process_data["props"]["$name"] + " (" + protocol_process_data["registration_date"] + ")"
            self.accordion.set_title(len(accordion_items), process_step_title)
            accordion_items.append(protocol_process_accordion)
        self.accordion.children = accordion_items
        
    def add_process_step(self, b):
        # Process Step properties
        properties_widgets = widgets.ObjectPropertiesWidgets("SampleProcess")
        properties_widgets.get_properties_widgets()
        
        # Actions
        actions_widgets = ActionsWidgets(history = False)
        
        # Observations
        observables_widgets = ObservablesWidgets(history = False)
        
        # Process Step widgets
        process_step_widgets = ProcessStepWidgets()
        process_step_widgets.set_process_step(None, actions_widgets, observables_widgets)
        
        remove_process_step_button = ipw.Button(description = "Remove process step", button_style = 'danger')
        process_step_items = ipw.VBox([properties_widgets, process_step_widgets, remove_process_step_button])
        
        def update_item_title(change):
            item_idx, _, process_type = self.accordion_items[process_step_items]
            new_title = properties_widgets.properties_widgets_detailed_dict["$name"].value
            self.accordion.set_title(item_idx, new_title)
            self.accordion_items[process_step_items] = [item_idx, new_title, process_type]
        
        properties_widgets.properties_widgets_detailed_dict["$name"].observe(update_item_title, names = 'value')
        
        def remove_process_step(b):
            self.accordion_items.pop(process_step_items)
            i = 0
            for item, item_info in self.accordion_items.items():
                self.accordion_items[item] = [i, item_info[1], item_info[2]]
                self.accordion.set_title(i, item_info[1])
                i += 1
            self.accordion.set_title(i, "")
            self.accordion.children = list(self.accordion_items.keys())
        
        remove_process_step_button.on_click(remove_process_step)
        self.accordion_items[process_step_items] = [len(self.accordion_items), "", "Process step"]
        self.accordion.children = list(self.accordion_items.keys())
    
    def add_protocol_process(self, b):
        # Process properties
        properties_widgets = widgets.ObjectPropertiesWidgets("SampleProcess")
        properties_widgets.get_properties_widgets()
        
        protocol_label = ipw.HTML("<b><span style='font-size:14px;'>Protocol</span></b>")
        protocol_selector_config = {
            "dropdown": {"width": "600px"}
        }
        protocol_selector = widgets.ObjectSelectionWidget("Protocol", protocol_selector_config)
        protocol_selector.load_dropdown_box()
        
        protocol_process_widgets = ProtocolProcessWidgets()
        
        def add_protocol_process_steps(change):
            protocol_identifier = protocol_selector.dropdown.value
            protocol_object = utils.get_openbis_object_data(OPENBIS_SESSION, protocol_identifier)
            protocol_template = protocol_object["props"]["protocol_template"]
            protocol_template = json.loads(protocol_template)
            protocol_process_widgets.set_protocol_process_steps(protocol_template)
        
        protocol_selector.dropdown.observe(add_protocol_process_steps, names = "value")
        remove_protocol_process_button = ipw.Button(description = "Remove process", button_style = 'danger')
        protocol_process_items = ipw.VBox([properties_widgets, protocol_label, protocol_selector, 
                                             protocol_process_widgets, remove_protocol_process_button])
        
        def update_item_title(change):
            item_idx, _, process_type = self.accordion_items[protocol_process_items]
            new_title = properties_widgets.properties_widgets_detailed_dict["$name"].value
            self.accordion.set_title(item_idx, new_title)
            self.accordion_items[protocol_process_items] = [item_idx, new_title, process_type]
        
        properties_widgets.properties_widgets_detailed_dict["$name"].observe(update_item_title, names = 'value')
        
        def remove_protocol_process(b):
            self.accordion_items.pop(protocol_process_items)
            i = 0
            for item, item_info in self.accordion_items.items():
                self.accordion_items[item] = [i, item_info[1], item_info[2]]
                self.accordion.set_title(i, item_info[1])
                i += 1
            self.accordion.set_title(i, "")
            self.accordion.children = list(self.accordion_items.keys())
        
        remove_protocol_process_button.on_click(remove_protocol_process)
        self.accordion_items[protocol_process_items] = [len(self.accordion_items), "", "Protocol process"]
        self.accordion.children = list(self.accordion_items.keys())
        
def get_sample_processes(sample_identifier):
    sample_processes = {}
    sample_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = sample_identifier)
    objects_stack = [(sample_object, None)]
    
    while objects_stack:
        current_object, child_object = objects_stack.pop()
        current_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, current_object.permid)
        
        if current_object_data["type"] == "SAMPLE_PROCESS" and current_object_data["permId"] not in sample_processes:
            process_actions = []
            process_observables = []
            process_children = current_object_data["children"]
            for child_identifier in process_children:
                child_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, child_identifier)
                if child_object_data["type"] == "ACTION":
                    process_actions.append(child_object_data["permId"])
                elif child_object_data["type"] == "OBSERVABLE":
                    process_observables.append(child_object_data["permId"])
            
            if process_actions or process_observables:   
                sample_processes[current_object_data["permId"]] = {
                    "actions": process_actions,
                    "observables": process_observables
                }
        
        for parent_identifier in current_object.parents:
            parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent_identifier)
            objects_stack.append((parent_object, current_object))
            
    return sample_processes

In [ ]:
sample_selector = widgets.ObjectSelectionWidget("Sample")
sample_selector.load_dropdown_box()

sample_preparation_history = SamplePreparationWidgets(history = True)

add_process_button = utils.Button(
    description = 'Add protocol process', disabled = False, button_style = 'success', 
    tooltip = 'Add process with protocol', layout = ipw.Layout(width = '150px', height = '25px')
)

add_process_step_button = utils.Button(
    description = 'Add process step', disabled = False, button_style = 'success', 
    tooltip = 'Add process step', layout = ipw.Layout(width = '150px', height = '25px')
)

add_processes_buttons = ipw.HBox([add_process_button, add_process_step_button])

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
new_sample_processes = SamplePreparationWidgets(history = False)

In [ ]:

def load_sample_metadata(change):
    if sample_selector.dropdown.value == -1:
        sample_preparation_history.children = []
    else:
        sample_processes = get_sample_processes(sample_selector.dropdown.value)
        sample_preparation_history.create_process_step_widgets(OPENBIS_SESSION, sample_processes)
        sample_preparation_history.selected_index = None

def save_processes(b):
    if sample_selector.dropdown.value == -1:
        print("Select a sample.")
    else:
        for item, item_info in new_sample_processes.accordion_items.items():
            if item_info[2] == "Process step":
                # Process step
                process_step_properties_widgets = item.children[0]
                process_step_properties = utils.get_widget_values(process_step_properties_widgets.properties_widgets_detailed_dict)
                
                # Actions
                process_step_widgets = item.children[1]
                process_step_actions = process_step_widgets.children[1]
                process_step_actions_accordion = process_step_actions.children[0]
                process_step_actions_accordion_items = process_step_actions_accordion.children
                for action in process_step_actions_accordion_items:
                    action_properties_widgets = action.children[1]
                    action_properties = utils.get_widget_values(action_properties_widgets.properties_widgets_detailed_dict)
                
                # Observables
                process_step_observables = item.children[1].children[3].children[0].children
                process_step_observables = process_step_widgets.children[3]
                process_step_observables_accordion = process_step_observables.children[0]
                process_step_observables_accordion_items = process_step_observables_accordion.children
                for observable in process_step_observables_accordion_items:
                    observable_properties_widgets = observable.children[1]
                    observable_properties = utils.get_widget_values(observable_properties_widgets.properties_widgets_detailed_dict)
                    
            elif item_info[2] == "Protocol process":
                # Protocol process
                protocol_process_properties_widgets = item.children[0]
                protocol_process_properties = utils.get_widget_values(protocol_process_properties_widgets.properties_widgets_detailed_dict)
                
                # Protocol identifier
                protocol_identifier = item.children[2].dropdown.value
                
                # Create protocol process in openBIS
                protocol_process_openbis = OPENBIS_SESSION.new_object(
                    type = "SAMPLE_PROCESS", experiment = "/DEFAULT/DEFAULT/DEFAULT_EXP_1",
                    props = protocol_process_properties, parents = [protocol_identifier])
                protocol_process_openbis.save()
                
                # Protocol process steps
                protocol_process_steps = item.children[3].accordion_items
                
                for item, item_info in protocol_process_steps.items():
                    # Process step
                    process_step_properties_widgets = item.children[0]
                    process_step_properties = utils.get_widget_values(process_step_properties_widgets.properties_widgets_detailed_dict)
                    
                    # Create protocol process in openBIS
                    process_step_openbis = OPENBIS_SESSION.new_object(
                        type = "SAMPLE_PROCESS", experiment = "/DEFAULT/DEFAULT/DEFAULT_EXP_1",
                        props = process_step_properties, parents = [protocol_process_openbis])
                    process_step_openbis.save()
                    
                    # Actions
                    process_step_widgets = item.children[1]
                    process_step_actions = process_step_widgets.children[1]
                    process_step_actions_accordion = process_step_actions.children[0]
                    process_step_actions_accordion_items = process_step_actions_accordion.children
                    for action in process_step_actions_accordion_items:
                        action_properties_widgets = action.children[1]
                        action_properties = utils.get_widget_values(action_properties_widgets.properties_widgets_detailed_dict)
                    
                    # Observables
                    process_step_observables = item.children[1].children[3].children[0].children
                    process_step_observables = process_step_widgets.children[3]
                    process_step_observables_accordion = process_step_observables.children[0]
                    process_step_observables_accordion_items = process_step_observables_accordion.children
                    for observable in process_step_observables_accordion_items:
                        observable_properties_widgets = observable.children[1]
                        observable_properties = utils.get_widget_values(observable_properties_widgets.properties_widgets_detailed_dict)

                


def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

In [ ]:
display(Markdown("## Select sample"))
sample_selector.dropdown.observe(load_sample_metadata, names = 'value')
display(sample_selector)

display(Markdown("## Sample history"))
display(sample_preparation_history)

display(Markdown("## Register new sample processing steps"))
display(new_sample_processes)
create_button.on_click(save_processes)
quit_button.on_click(close_notebook)
display(save_close_buttons_hbox)
display(increase_buttons_size)